In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt 
import h5py
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import scipy

def GetLP(Frames, num):
    temp = 0
    Lowest = [] 
    Id = []
    for i in range(Frames.shape[0]):
        if i == 0:
            temp = GetPWR(Frames[i,0,:])

        p = GetPWR(Frames[i,0,:])
        if p < temp:
            temp = p 
            Lowest.append(temp)
            Id.append(i)
            #plt.plot(Frames[i,:,0])
            #plt.show()
            #print(p)
            
    Lowst = np.array(Lowest)
    l50 = np.sort(Lowst)[:num]
    print(f"The lowest power is {temp}\nAnd the lowest 50 are {l50}")
    print("===========================================================")
    return l50, Id
    
def GetPWR(signal):
    Pwr = np.sum(signal**2)
    return Pwr

def Ifilter(Frames, Ref_Pwr):
    Del_inx = []
    d = 0
    for i in range(Frames.shape[0]):
        tmp_pwr = GetPWR(Frames[i,:,0])
        if tmp_pwr <= Ref_Pwr:
            Del_inx.append(i)
            d += 1
            #print("detect index", i)

    CFrames = np.delete(Frames, Del_inx, axis=0)
    print("Detect ", d, "Bad Frames :(")
    return CFrames

def Qfilter(Frames, Ref_Pwr):
    Lowest = Ref_Pwr
    Highest = 0
    Bad_inx = []
    for i in range(Frames.shape[0]):
        p = GetPWR(Frames[i,:,1])
        if p < Lowest:
            print("Ops")
            Bad_inx.append(i)
            
        if p > Highest:
            Highest = p
            #x = i
    CFrames = np.delete(Frames, Bad_inx, axis=0)
    return CFrames

def Efilter(Frames, Th):
    X = np.random.choice(Frames.shape[0], int(Frames.shape[0]))
    Ps = []
    for x in X:
        p = np.around(GetPWR(Frames[x,0]), 2)
        Ps.append(p)

    Avg_power = scipy.stats.mode(Ps)[0]
        
    #Powermax ==> loc1: 290, loc2: 42, loc3: 087,
    Uhave = Frames.shape[0]
    Bad_inx = []
    Bad2_inx = []
    Bad3_inx = []
    for i in range(Frames.shape[0]):
        sig = Frames[i][0]
        if np.all(np.abs(sig[sig.shape[0]-50:]) < Th):
            Bad_inx.append(i)

    Frames = np.delete(Frames, Bad_inx, axis=0)
    ###################################################
    for i in range(Frames.shape[0]):
        for j in range(int(Frames.shape[0]/500)):  
            sig = Frames[i][0][j*20: j*20 + 20]
            if np.all(np.abs(sig) < Th):
                Bad2_inx.append(i)

    Frames = np.delete(Frames, list(set(Bad2_inx)), axis=0)
    ####################################################
    for i in range(Frames.shape[0]):
        sig = Frames[i][0]
        '''
        if np.all(np.abs(sig[:20]): < (Th)):# or GetPWR(sig) > 1.2*Avg_power or GetPWR(sig) < Avg_power/1.185:
            Bad3_inx.append(i)

        '''
        if np.all(np.abs(sig[:20]) < (Th)):# or GetPWR(sig) > 1.2*Avg_power or GetPWR(sig) < Avg_power/1.5:
            Bad3_inx.append(i)
    Frames = np.delete(Frames, Bad3_inx, axis=0)

    bad = len(Bad_inx)+len(set(Bad2_inx))+len(Bad3_inx)
    print("Detect", bad, "Frames, and now you've got", Uhave-bad)        

    return Frames

In [12]:
#loading the train_data
loc = "Ch2"
root_dir = f"BLE_Data/Data_Sep_3/{loc}"
enc = OneHotEncoder()
num_samples = 2400
num_tests = 400
class_label = []
Y_train = []
X_train = []
Y_test =[]
X_test = []
j = 0
for class_dir in sorted(os.listdir(root_dir)):
    class_dir = os.path.join(root_dir, class_dir)
    label = enc.fit_transform(np.array(sorted(os.listdir(root_dir))).reshape(-1,1)).toarray()[j]
    class_label.append(np.hstack((class_dir,label)))
    #label = enc.fit_transform(np.array(os.listdir(root_dir)).reshape(-1,1))[j]
    j += 1 
    print(class_dir, label)
    
    x = np.load(class_dir)
    x = Efilter(x, 0.07) #1357
    x = Efilter(x, 0.015)
    if x.shape[0] < num_samples:
        rnd_inds = np.random.choice(x.shape[0], x.shape[0], replace=False)
        t_inds = np.random.choice(x.shape[0] - int(num_tests/2), int(num_tests/2), replace=False)

    else:
        rnd_inds = np.random.choice(x.shape[0], num_samples, replace=False)
        t_inds = np.random.choice(num_samples - num_tests, num_tests, replace=False)
        
        
    x = x[rnd_inds]
    Xt = x[t_inds]
    
    x = np.delete(x, t_inds, axis=0)
    
    y = np.ones(shape=[x.shape[0], 1]) * label
    Yt = np.ones(shape=[Xt.shape[0],1]) * label
    
    X_train.append(x)
    Y_train.append(y)

    X_test.append(Xt)
    Y_test.append(Yt)
    print("=====================\nyou got", len(x), "for train and", len(Xt),"for test\n============== done =================")

X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)

X_test = np.vstack(X_test)
Y_test = np.vstack(Y_test)

np.save(f"BLE_Raw/RawIQ_Sep_3/{loc}/X_train.npy", X_train)
np.save(f"BLE_Raw/RawIQ_Sep_3/{loc}/Y_train.npy", Y_train)
np.save(f"BLE_Raw/RawIQ_Sep_3/{loc}/X_test.npy", X_test)
np.save(f"BLE_Raw/RawIQ_Sep_3/{loc}/Y_test.npy", Y_test)

BLE_Data/Data_Sep_3/Ch2/Dev1.npy [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Detect 0 Frames, and now you've got 3267
Detect 0 Frames, and now you've got 3267
you got 2000 for train and 400 for test
============== done =================
BLE_Data/Data_Sep_3/Ch2/Dev10.npy [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Detect 1182 Frames, and now you've got 3367
Detect 0 Frames, and now you've got 3367
you got 2000 for train and 400 for test
============== done =================
BLE_Data/Data_Sep_3/Ch2/Dev11.npy [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Detect 5 Frames, and now you've got 4016
Detect 0 Frames, and now you've got 4016
you got 2000 for train and 400 for test
============== done =================
BLE_Data/Data_Sep_3/Ch2/Dev12.npy [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.

In [5]:
#loading the train_data
loc = 4
root_dir = f"Data_31_Devices_Renamed/loc{loc}"
enc = OneHotEncoder()
num_samples = 2400
num_tests = 400
class_label = []
Y_train = []
X_train = []
Y_test =[]
X_test = []
j = 0
for class_dir in sorted(os.listdir(root_dir)):
    class_dir = os.path.join(root_dir, class_dir)
    label = enc.fit_transform(np.array(sorted(os.listdir(root_dir))).reshape(-1,1)).toarray()[j]
    class_label.append(np.hstack((class_dir,label)))
    #label = enc.fit_transform(np.array(os.listdir(root_dir)).reshape(-1,1))[j]
    j += 1 
    print(class_dir, label)
    
    x = np.load(class_dir)
    x = Efilter(x, 0.025)
    x = Efilter(x, 0.015)
    if x.shape[0] < num_samples:
        rnd_inds = np.random.choice(x.shape[0], x.shape[0], replace=False)
        t_inds = np.random.choice(x.shape[0] - int(num_tests/2), int(num_tests/2), replace=False)

    else:
        rnd_inds = np.random.choice(x.shape[0], num_samples, replace=False)
        t_inds = np.random.choice(num_samples - num_tests, num_tests, replace=False)
        
        
    x = x[rnd_inds]
    Xt = x[t_inds]
    
    x = np.delete(x, t_inds, axis=0)
    
    y = np.ones(shape=[x.shape[0], 1]) * label
    Yt = np.ones(shape=[Xt.shape[0],1]) * label
    
    X_train.append(x)
    Y_train.append(y)

    X_test.append(Xt)
    Y_test.append(Yt)
    print("=====================\nyou got", len(x), "for train and", len(Xt),"for test\n============== done =================")

X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)

X_test = np.vstack(X_test)
Y_test = np.vstack(Y_test)

np.save(f"RawIQ_31_Devices_Renamed/X_train_{loc}.npy", X_train)
np.save(f"RawIQ_31_Devices_Renamed/Y_train_{loc}.npy", Y_train)
np.save(f"RawIQ_31_Devices_Renamed/X_test_{loc}.npy", X_test)
np.save(f"RawIQ_31_Devices_Renamed/Y_test_{loc}.npy", Y_test)

Data_31_Devices_Renamed/loc4/Dev1.npy [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Detect 4 Frames, and now you've got 2650
Detect 0 Frames, and now you've got 2650
you got 2000 for train and 400 for test
============== done =================
Data_31_Devices_Renamed/loc4/Dev10.npy [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Detect 0 Frames, and now you've got 2309
Detect 0 Frames, and now you've got 2309
you got 2109 for train and 200 for test
============== done =================
Data_31_Devices_Renamed/loc4/Dev11.npy [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Detect 9 Frames, and now you've got 3190
Detect 0 Frames, and now you've got 3190
you got 2000 for train and 400 for test
============== done =================
Data_31_Devices_Renamed/loc4/Dev12.npy [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 